In [ ]:
%sc

!wget 'https://s3-eu-west-1.amazonaws.com/crimedatawalker/crime.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
crime = pd.read_csv('crime.csv', header=0)
print(crime.head())



In [ ]:
def stateToNumber(s):
    l = 0
    for x in s:
        l = l + int(hex(ord(x)),16)
    return l

In [ ]:


xref = pd.DataFrame(crime['State'])

crime['State']=crime['State'].apply(lambda x: stateToNumber(x))

crime.head()

In [ ]:
crimeArray = crime.as_matrix().astype(np.float32)


In [ ]:
crimeArray.shape

In [ ]:
from sagemaker import KMeans
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

bucket = "sagemakerwalkerml"
data_location = "sagemakerwalkerml"


data_location = 's3://{}/kmeans_highlevel_example/data'.format(bucket)
output_location = 's3://{}/kmeans_example/output'.format(bucket)

print('training data will be uploaded to: {}'.format(data_location))
print('training artifacts will be uploaded to: {}'.format(output_location))

kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.8xlarge',
                output_path=output_location,
                k=10,
                data_location=data_location)

In [ ]:
slice=crimeArray[:,1:5]

In [339]:
%%time
kmeans.fit(kmeans.record_set(slice)) 

...........................................................
Docker entrypoint called with argument(s): train
[03/28/2018 14:17:51 INFO 140290884646720] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local_lloyd_max_iter': u'300', u'_disable_wait_to_read': u'false', u'extra_center_factor': u'auto', u'eval_metrics': u'["msd"]', u'_num_kv_servers': u'1', u'mini_batch_size': u'5000', u'half_life_time_size': u'0', u'_num_slices': u'1'}
[03/28/2018 14:17:51 INFO 140290884646720] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'4', u'mini_batch_size': u'50', u'k': u'10', u'force_dense': u'True'}
[03/28/2018 14:17:51 INFO 140290884646720] Final

===== Job Complete =====
CPU times: user 548 ms, sys: 16 ms, total: 564 ms
Wall time: 8min 9s


In [340]:
%%time

kmeans_predictor = kmeans.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: kmeans-2018-03-28-14-19-45-414
INFO:sagemaker:Creating endpoint with name kmeans-2018-03-28-14-11-35-745


----------------------------------------------------------------------------------------------------------------------------!CPU times: user 548 ms, sys: 28 ms, total: 576 ms
Wall time: 10min 29s


In [341]:
slice=crimeArray[:,1:5]
slice.shape
slice

array([[   4.        ,   13.19999981,  236.        ,   58.        ],
       [   4.        ,   10.        ,  263.        ,   48.        ],
       [   4.        ,    8.10000038,  294.        ,   80.        ],
       [   3.        ,    8.80000019,  190.        ,   50.        ],
       [   4.        ,    9.        ,  276.        ,   91.        ],
       [   3.        ,    7.9000001 ,  204.        ,   78.        ],
       [   2.        ,    3.29999995,  110.        ,   77.        ],
       [   4.        ,    5.9000001 ,  238.        ,   72.        ],
       [   4.        ,   15.39999962,  335.        ,   80.        ],
       [   3.        ,   17.39999962,  211.        ,   60.        ],
       [   1.        ,    5.30000019,   46.        ,   83.        ],
       [   2.        ,    2.5999999 ,  120.        ,   54.        ],
       [   4.        ,   10.39999962,  249.        ,   83.        ],
       [   2.        ,    7.19999981,  113.        ,   65.        ],
       [   1.        ,    2.200000

In [342]:
s=slice[1:2]

In [343]:
%%time

result = kmeans_predictor.predict(s)
clusters = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]
i = 0
 
    
for r in result:
    out = {
      "State" : crime['State'].iloc[i],  
      "StateCode" : xref['State'].iloc[i],  
      "closest_cluster" :  r.label['closest_cluster'].float32_tensor.values[0],
      "crimeCluster" : crime['crimeCluster'].iloc[i],  
      "Murder" :  crime['Murder'].iloc[i],  
      "Assault" : crime['Assault'].iloc[i], 
      "UrbanPop" : crime['UrbanPop'].iloc[i],  
      "Rape" :  crime['Rape'].iloc[i]
    }    
    print(out) 
    i = i + 1


 

{'State': 671, 'StateCode': 'Alabama', 'closest_cluster': 7.0, 'crimeCluster': 4, 'Murder': 13.199999999999999, 'Assault': 236, 'UrbanPop': 58, 'Rape': 21.199999999999999}
CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 298 ms


In [344]:
%%time

result = kmeans_predictor.predict(slice)
clusters = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]
i = 0
 
    
for r in result:
    out = {
      "State" : crime['State'].iloc[i],  
      "StateCode" : xref['State'].iloc[i],  
      "closest_cluster" :  r.label['closest_cluster'].float32_tensor.values[0],
      "crimeCluster" : crime['crimeCluster'].iloc[i],  
      "Murder" :  crime['Murder'].iloc[i],  
      "Assault" : crime['Assault'].iloc[i], 
      "UrbanPop" : crime['UrbanPop'].iloc[i],  
      "Rape" :  crime['Rape'].iloc[i]
    }    
    print(out) 
    i = i + 1

{'State': 671, 'StateCode': 'Alabama', 'closest_cluster': 1.0, 'crimeCluster': 4, 'Murder': 13.199999999999999, 'Assault': 236, 'UrbanPop': 58, 'Rape': 21.199999999999999}
{'State': 589, 'StateCode': 'Alaska', 'closest_cluster': 7.0, 'crimeCluster': 4, 'Murder': 10.0, 'Assault': 263, 'UrbanPop': 48, 'Rape': 44.5}
{'State': 724, 'StateCode': 'Arizona', 'closest_cluster': 3.0, 'crimeCluster': 4, 'Murder': 8.0999999999999996, 'Assault': 294, 'UrbanPop': 80, 'Rape': 31.0}
{'State': 820, 'StateCode': 'Arkansas', 'closest_cluster': 6.0, 'crimeCluster': 3, 'Murder': 8.8000000000000007, 'Assault': 190, 'UrbanPop': 50, 'Rape': 19.5}
{'State': 1016, 'StateCode': 'California', 'closest_cluster': 3.0, 'crimeCluster': 4, 'Murder': 9.0, 'Assault': 276, 'UrbanPop': 91, 'Rape': 40.600000000000001}
{'State': 819, 'StateCode': 'Colorado', 'closest_cluster': 6.0, 'crimeCluster': 3, 'Murder': 7.9000000000000004, 'Assault': 204, 'UrbanPop': 78, 'Rape': 38.700000000000003}
{'State': 1151, 'StateCode': 'Conn